<a href="https://colab.research.google.com/github/neryguilherme/thegamelibrary/blob/Verft4-patch-1/Classifica%C3%A7%C3%A3o_Mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Carregar os dados
banco = pd.read_parquet('/content/games.parquet')

# Reduzir o dataset para 1000 amostras
if len(banco) > 35000:
    banco = banco.sample(n=35000, random_state=42)

banco

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5982,840810,Data Ball,"Jun 26, 2018",0 - 20000,0,0,9.99,0,The Style Remember when games were simpler? Wh...,['English'],...,0,0,0,Munkyfoot Gaming,Munkyfoot Gaming,"Single-player,Multi-player,PvP,Online PvP,Shar...","Casual,Indie,Racing,Strategy,Early Access","Strategy,Indie,Casual,Racing,Early Access,3D P...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
30861,526400,Embers of Magic,"Apr 6, 2017",20000 - 50000,0,0,0.00,0,"Embers of Magic is a short, adventure visual n...",['English'],...,0,0,0,Ithaqua Labs,Ithaqua Labs,"Single-player,Steam Achievements","Action,Adventure,Casual,Free to Play,Indie","Free to Play,Indie,Visual Novel,Casual,Adventu...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
67876,2258240,Factory Town Idle Playtest,"Dec 26, 2022",0 - 0,0,0,0.00,0,None,[],...,0,0,0,None,None,None,None,None,None,None
55017,1149800,Xenrai,"Dec 22, 2021",0 - 20000,0,0,29.99,1,Save the last world in a solar system of livin...,"['English', 'Japanese']",...,0,0,0,Zenith Edge Studios,Zenith Edge Studios,"Single-player,Steam Achievements,Full controll...","Action,RPG,Early Access","Early Access,Spectacle fighter,Hack and Slash,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
59172,1637110,MoneyFalls - Coin Pusher Simulator,"Jun 25, 2021",0 - 20000,0,0,11.99,0,MoneyFalls is a coin pusher simulator with acc...,['English'],...,0,0,0,Mako 6 fingers,Digital Picklock,Single-player,"Casual,Simulation","Casual,Simulation,Arcade,3D,Immersive Sim,Firs...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57397,387430,The Viceroy,"Jul 28, 2015",0 - 20000,0,0,19.99,0,OVERVIEW The Viceroy is a turn-based economic ...,['English'],...,0,0,0,Goatee Games,Goatee Games,"Single-player,Steam Achievements,Steam Cloud","Indie,Simulation,Strategy","Strategy,Simulation,Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
19463,785940,Strategic Command Classic: Global Conflict,"Feb 8, 2018",0 - 20000,0,0,29.99,0,The Matrix version is the Gold Edition of the ...,['English'],...,0,0,0,Fury Software,Slitherine Ltd.,"Single-player,Multi-player,Shared/Split Screen...",Strategy,Strategy,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
5694,3340,AstroPop Deluxe,"Aug 30, 2006",20000 - 50000,6,0,4.99,0,In a world where most puzzle games are the sam...,['English'],...,0,0,0,"PopCap Games, Inc.","PopCap Games, Inc.",Single-player,Casual,"Casual,Puzzle",https://cdn.akamai.steamstatic.com/steam/apps/...,None
50981,1447710,Death By FrostByte,"Mar 10, 2021",0 - 20000,0,0,9.99,1,"After years of scorching heat, an incoming fro...",['English'],...,0,0,0,Stormee Games,Stormee Games,"Single-player,Steam Achievements,Steam Cloud","Casual,Indie,Strategy,Early Access","Early Access,Strategy,Management,Survival,Time...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [ ]:
# Preenchimento de valores faltantes
for col in banco.columns:
    if banco[col].dtype == 'object':
        banco[col] = banco[col].fillna(banco[col].mode()[0])  # Preenche com a moda
    elif pd.api.types.is_numeric_dtype(banco[col]):
        banco[col] = banco[col].fillna(banco[col].median())  # Preenche com a mediana
banco


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5982,840810,Data Ball,"Jun 26, 2018",0 - 20000,0,0,9.99,0,The Style Remember when games were simpler? Wh...,['English'],...,0,0,0,Munkyfoot Gaming,Munkyfoot Gaming,"Single-player,Multi-player,PvP,Online PvP,Shar...","Casual,Indie,Racing,Strategy,Early Access","Strategy,Indie,Casual,Racing,Early Access,3D P...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
30861,526400,Embers of Magic,"Apr 6, 2017",20000 - 50000,0,0,0.00,0,"Embers of Magic is a short, adventure visual n...",['English'],...,0,0,0,Ithaqua Labs,Ithaqua Labs,"Single-player,Steam Achievements","Action,Adventure,Casual,Free to Play,Indie","Free to Play,Indie,Visual Novel,Casual,Adventu...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
67876,2258240,Factory Town Idle Playtest,"Dec 26, 2022",0 - 0,0,0,0.00,0,Help the beautiful girls to immerse themselves...,[],...,0,0,0,Choice of Games,Big Fish Games,Single-player,"Casual,Indie","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
55017,1149800,Xenrai,"Dec 22, 2021",0 - 20000,0,0,29.99,1,Save the last world in a solar system of livin...,"['English', 'Japanese']",...,0,0,0,Zenith Edge Studios,Zenith Edge Studios,"Single-player,Steam Achievements,Full controll...","Action,RPG,Early Access","Early Access,Spectacle fighter,Hack and Slash,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
59172,1637110,MoneyFalls - Coin Pusher Simulator,"Jun 25, 2021",0 - 20000,0,0,11.99,0,MoneyFalls is a coin pusher simulator with acc...,['English'],...,0,0,0,Mako 6 fingers,Digital Picklock,Single-player,"Casual,Simulation","Casual,Simulation,Arcade,3D,Immersive Sim,Firs...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57397,387430,The Viceroy,"Jul 28, 2015",0 - 20000,0,0,19.99,0,OVERVIEW The Viceroy is a turn-based economic ...,['English'],...,0,0,0,Goatee Games,Goatee Games,"Single-player,Steam Achievements,Steam Cloud","Indie,Simulation,Strategy","Strategy,Simulation,Indie",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
19463,785940,Strategic Command Classic: Global Conflict,"Feb 8, 2018",0 - 20000,0,0,29.99,0,The Matrix version is the Gold Edition of the ...,['English'],...,0,0,0,Fury Software,Slitherine Ltd.,"Single-player,Multi-player,Shared/Split Screen...",Strategy,Strategy,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
5694,3340,AstroPop Deluxe,"Aug 30, 2006",20000 - 50000,6,0,4.99,0,In a world where most puzzle games are the sam...,['English'],...,0,0,0,"PopCap Games, Inc.","PopCap Games, Inc.",Single-player,Casual,"Casual,Puzzle",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
50981,1447710,Death By FrostByte,"Mar 10, 2021",0 - 20000,0,0,9.99,1,"After years of scorching heat, an incoming fro...",['English'],...,0,0,0,Stormee Games,Stormee Games,"Single-player,Steam Achievements,Steam Cloud","Casual,Indie,Strategy,Early Access","Early Access,Strategy,Management,Survival,Time...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [ ]:
# Separar a coluna 'Name'
if 'Name' in banco.columns:
    name_column = banco['Name']
    x_column = banco.drop(columns=['Name'])
else:
    raise ValueError("A coluna 'Name' não foi encontrada no dataset.")

In [ ]:
# prompt: aplique o label encoder nos atributos

# Aplicar Label Encoding aos atributos categóricos
label_encoders = {}
for column in x_column.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    x_column[column] = le.fit_transform(x_column[column])
    label_encoders[column] = le


In [ ]:

# Escalonar os dados numéricos
scaler = StandardScaler()
numerical_cols = x_column.select_dtypes(include=['number']).columns
x_column[numerical_cols] = scaler.fit_transform(x_column[numerical_cols])
x_column


,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5982,-0.597299,0.130170,-0.436017,-0.021854,-0.145896,0.276637,-0.077669,0.911488,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.201126,0.226360,0.094258,0.758609,1.537084,1.416857,-0.752434
30861,-1.123783,-1.488501,1.208797,-0.021854,-0.145896,-0.706808,-0.077669,-0.773228,0.567506,-0.271204,...,-0.06087,-0.060388,-0.057539,-0.304866,-0.271389,0.889848,-1.739445,0.067303,0.825164,-1.394538
67876,1.776209,-0.992970,-0.847220,-0.021854,-0.145896,-0.706808,-0.077669,-0.398255,0.739115,0.555430,...,-0.06087,-0.060388,-0.057539,-1.152702,-1.299191,-1.047627,0.665022,0.346647,1.243828,-0.227756
55017,-0.079891,-1.022009,-0.436017,-0.021854,-0.145896,2.245495,0.066736,0.544973,-0.313612,-1.571775,...,-0.06087,-0.060388,-0.057539,1.494712,1.508971,0.918262,-0.342838,-0.134178,-1.455747,0.490026
59172,0.736117,0.124550,-0.436017,-0.021854,-0.145896,0.473523,-0.077669,0.102440,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.050236,-0.888630,-1.047627,0.866594,-0.440676,-0.610987,0.571652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57397,-1.356491,-0.135861,-0.436017,-0.021854,-0.145896,1.261066,-0.077669,0.188482,0.567506,-0.271204,...,-0.06087,-0.060388,-0.057539,-0.539487,-0.508067,1.116271,1.406519,1.612396,0.576918,-1.594473
19463,-0.689180,-0.633265,-0.436017,-0.021854,-0.145896,2.245495,-0.077669,0.886949,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,-0.633595,0.911837,0.718476,1.788066,1.444219,1.310710,-0.946971
5694,-1.999655,-1.240267,1.208797,-0.020981,-0.145896,-0.215578,-0.077669,-0.241729,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.523081,0.550062,-1.047627,0.456251,-0.506687,0.468762,-0.227756
50981,0.418964,0.246325,-0.436017,-0.021854,-0.145896,0.276637,0.066736,-1.380849,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.988432,1.009244,1.116271,0.799403,-0.128327,-0.938266,0.988596


In [ ]:
# prompt: Faça a variancia e remova os atributos inuteis e pront as colunas inuteis

# Calcular a variância de cada coluna
variances = x_column.var()

# Definir um limiar de variância (ex: 0.01, que representa 1%)
threshold = 0.01

# Identificar colunas com variância abaixo do limiar
low_variance_cols = variances[variances < threshold].index.tolist()

# Remover as colunas de baixa variância
x_column = x_column.drop(columns=low_variance_cols)

# Imprimir as colunas que foram removidas
print("Colunas removidas devido à baixa variância:")
print(low_variance_cols)

# Remover colunas com apenas um valor único
unique_value_cols = [col for col in x_column.columns if x_column[col].nunique() == 1]
x_column = x_column.drop(columns=unique_value_cols)
print("Colunas removidas com valor único:")
print(unique_value_cols)
x_column


Colunas removidas devido à baixa variância:
['Windows']
Colunas removidas com valor único:
[]


,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5982,-0.597299,0.130170,-0.436017,-0.021854,-0.145896,0.276637,-0.077669,0.911488,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.201126,0.226360,0.094258,0.758609,1.537084,1.416857,-0.752434
30861,-1.123783,-1.488501,1.208797,-0.021854,-0.145896,-0.706808,-0.077669,-0.773228,0.567506,-0.271204,...,-0.06087,-0.060388,-0.057539,-0.304866,-0.271389,0.889848,-1.739445,0.067303,0.825164,-1.394538
67876,1.776209,-0.992970,-0.847220,-0.021854,-0.145896,-0.706808,-0.077669,-0.398255,0.739115,0.555430,...,-0.06087,-0.060388,-0.057539,-1.152702,-1.299191,-1.047627,0.665022,0.346647,1.243828,-0.227756
55017,-0.079891,-1.022009,-0.436017,-0.021854,-0.145896,2.245495,0.066736,0.544973,-0.313612,-1.571775,...,-0.06087,-0.060388,-0.057539,1.494712,1.508971,0.918262,-0.342838,-0.134178,-1.455747,0.490026
59172,0.736117,0.124550,-0.436017,-0.021854,-0.145896,0.473523,-0.077669,0.102440,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.050236,-0.888630,-1.047627,0.866594,-0.440676,-0.610987,0.571652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57397,-1.356491,-0.135861,-0.436017,-0.021854,-0.145896,1.261066,-0.077669,0.188482,0.567506,-0.271204,...,-0.06087,-0.060388,-0.057539,-0.539487,-0.508067,1.116271,1.406519,1.612396,0.576918,-1.594473
19463,-0.689180,-0.633265,-0.436017,-0.021854,-0.145896,2.245495,-0.077669,0.886949,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,-0.633595,0.911837,0.718476,1.788066,1.444219,1.310710,-0.946971
5694,-1.999655,-1.240267,1.208797,-0.020981,-0.145896,-0.215578,-0.077669,-0.241729,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.523081,0.550062,-1.047627,0.456251,-0.506687,0.468762,-0.227756
50981,0.418964,0.246325,-0.436017,-0.021854,-0.145896,0.276637,0.066736,-1.380849,0.567506,0.555430,...,-0.06087,-0.060388,-0.057539,0.988432,1.009244,1.116271,0.799403,-0.128327,-0.938266,0.988596


In [ ]:
# Selecionar features (ajuste conforme necessário)
features = ['Tags', 'Categories', 'Price']
features = [f for f in features if f in x_column.columns]
X = x_column[features]


In [ ]:
y = le.fit_transform(name_column)
k = banco['Genres'] #Definindo k para a coluna Genres
k=le.fit_transform(k)


In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(x_column, k, test_size=0.3)

# Criar e treinar o modelo MLP
model = MLPClassifier(max_iter=800, verbose=True, tol=0.0000100,solver = 'adam',hidden_layer_sizes = (50,50),activation = 'relu',random_state=42)
model.fit(X_train, y_train)


# Fazer previsões e avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f"Acurácia do modelo: {accuracy}")

Iteration 1, loss = 5.60157795
Iteration 2, loss = 4.49796139
Iteration 3, loss = 4.09600232
Iteration 4, loss = 3.84932390
Iteration 5, loss = 3.65486843
Iteration 6, loss = 3.48360393
Iteration 7, loss = 3.32298718
Iteration 8, loss = 3.17457926
Iteration 9, loss = 3.04053967
Iteration 10, loss = 2.92120906
Iteration 11, loss = 2.81326350
Iteration 12, loss = 2.71726529
Iteration 13, loss = 2.63240224
Iteration 14, loss = 2.56129453
Iteration 15, loss = 2.48695407
Iteration 16, loss = 2.42463486
Iteration 17, loss = 2.36975838
Iteration 18, loss = 2.31825451
Iteration 19, loss = 2.27054666
Iteration 20, loss = 2.22343764
Iteration 21, loss = 2.18083635
Iteration 22, loss = 2.14711608
Iteration 23, loss = 2.11577769
Iteration 24, loss = 2.08019721
Iteration 25, loss = 2.04867795
Iteration 26, loss = 2.02392453
Iteration 27, loss = 1.99126994
Iteration 28, loss = 1.96477215
Iteration 29, loss = 1.93936280
Iteration 30, loss = 1.91394311
Iteration 31, loss = 1.89181941
Iteration 32, los

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           3       0.62      0.84      0.71       295
           4       0.47      0.26      0.33       139
           5       0.00      0.00      0.00        30
           7       0.20      0.33      0.25         3
           8       0.00      0.00      0.00         0
           9       0.21      0.21      0.21        14
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         2
          26       0.00      0.00      0.00         1
          27       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_